In [0]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text

In [0]:
data = open("train.txt","r")
data_test = open("test.txt","r")

In [0]:
def to_df(data):
    data=list(data)
    tweets = []
    sentiments = []
    arr = []
    for i in data:
        if(i=="\n"):
          tweets.append(arr)
          arr=[]
        else:
          temp=i.split("\t")
          if(len(temp)==3):
            sentiments.append(temp[2])
          arr.append(temp[0])
    sent = []
    for i in sentiments:
      if i == 'negative\n':
        sent = sent + [2]
      elif i == 'positive\n':
        sent = sent + [1]
      else:
        sent = sent + [0]       
    lis_twe = []
    for i in tweets:
      text = ''
      for j in i[1:]:
        if j.isalpha():
          text = text + ' ' + j
      lis_twe = lis_twe + [text]
    df = pd.DataFrame(list(zip(lis_twe, sent)),columns =['Tweets', 'Sentiments'])
    
    return df

In [0]:
df_train = to_df(data = data)
df_test = to_df(data = data_test)

In [0]:
df = pd.concat([df_train, df_test])

In [49]:
df.head()

,Tweets,Sentiments
0,AdilNisarButt pakistan ka ghra tauq he Pakist...,2
1,Madarchod mulle ye mathura me Nahi dikha tha ...,2
2,narendramodi Manya Pradhan Mantri mahoday Shr...,1
3,Atheist Krishna Jcb full trend me chal rahi aa,1
4,AbhisharSharma RavishKumarBlog Loksabha me ja...,1


In [0]:
x_train = df_train['Tweets'].values
y_train = df_train['Sentiments'].values

x_test = df_test['Tweets'].values
y_test = df_test['Sentiments'].values

classes = ['positive' , 'negative' , 'neutral']

In [51]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=classes,
                                                                       preprocess_mode='bert',
                                                                       maxlen=64, 
                                                                       max_features=40000)

preprocessing train...
language: id


preprocessing test...
language: id


In [52]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 64
done.


#Model Summary

In [56]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train),val_data=(x_test, y_test) ,batch_size = 32)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 64)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 64)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 64, 768), (1 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 64, 768)      1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

# Model Training

In [54]:
learner.autofit(3e-5, 5)



begin training using triangular learning rate policy with max lr of 3e-05...
Train on 15131 samples, validate on 1869 samples
Epoch 1/5
15131/15131 [==============================] - 162s 11ms/step - loss: 0.9185 - acc: 0.5376 - val_loss: 0.9589 - val_acc: 0.5650
Epoch 2/5
15131/15131 [==============================] - 144s 10ms/step - loss: 0.7895 - acc: 0.6423 - val_loss: 0.9731 - val_acc: 0.5634
Epoch 3/5
15131/15131 [==============================] - 144s 10ms/step - loss: 0.7020 - acc: 0.6948 - val_loss: 1.0027 - val_acc: 0.5666
Epoch 4/5
15131/15131 [==============================] - 144s 10ms/step - loss: 0.5980 - acc: 0.7503 - val_loss: 1.1089 - val_acc: 0.5618
Epoch 5/5
15131/15131 [==============================] - 145s 10ms/step - loss: 0.4766 - acc: 0.8128 - val_loss: 1.2526 - val_acc: 0.5511


#Model Testing

In [55]:
learner.validate(val_data=(x_test, y_test), class_names=classes)

              precision    recall  f1-score   support

    positive       0.55      0.46      0.50       754
    negative       0.55      0.64      0.59       582
     neutral       0.56      0.59      0.57       533

    accuracy                           0.55      1869
   macro avg       0.55      0.56      0.55      1869
weighted avg       0.55      0.55      0.55      1869



array([[345, 238, 171],
       [132, 370,  80],
       [155,  63, 315]])